In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image


In [ ]:
def residual_block(x, filters, kernel_size = 3):
  shortcut = x

  # First Convolution Layer
  x = layers.Conv2D(filters, kernel_size, padding="same")(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  # Second Convolution Layer
  x = layers.Conv2D(filters, kernel_size, padding="same")(x)
  x = layers.BatchNormalization()(x)

  # Add shortcut
  x = layers.Add()([x, shortcut])
  x = layers.ReLU()(x)

  return x

In [ ]:
def residual_block(x, filters, kernel_size=3):
    shortcut = x

    # Adjust shortcut if input channels ≠ output channels
    if x.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=1, padding="same")(shortcut)

    # Main path
    x = layers.Conv2D(filters, kernel_size, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, kernel_size, padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Add residual connection
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)

    return x


In [ ]:
model = build_resunet()
model.summary()

Model: "ResUNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 256, 256,  │        640 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_6 (ReLU)      │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 256, 256,  │     36,928 │ re_lu_6[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 256, 256,  │        128 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │ conv2d_8[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_7 (ReLU)      │ (None, 256, 256,  │          0 │ add_4[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 128, 128,  │          0 │ re_lu_7[0][0]     │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 128, 128,  │     73,856 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_12[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_8 (ReLU)      │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 128, 128,  │    147,584 │ re_lu_8[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_13[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 128, 128,  │      8,320 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 128, 128,  │          0 │ batch_normalizat

 Total params: 35,931,393 (137.07 MB)

 Trainable params: 35,919,617 (137.02 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='mean_squared_error',
    metrics=['mae']
)

In [ ]:
import zipfile
import os
from google.colab import files

# Step 1: Upload Sinogram ZIP
print("🔼 Upload your Sinogram ZIP (e.g. FILTERED_BRAIN_CT_SINOGRAMS.zip)")
sino_upload = files.upload()

# Step 2: Upload CT Image ZIP
print("\n🔼 Upload your CT Image ZIP (e.g. Brain_CT.zip)")
ct_upload = files.upload()

# Step 3: Extract both to correct folders
for fname in sino_upload:
    sino_folder = "/content/sinograms"
    os.makedirs(sino_folder, exist_ok=True)
    with zipfile.ZipFile(fname, 'r') as zip_ref:
        zip_ref.extractall(sino_folder)

for fname in ct_upload:
    ct_folder = "/content/ct_images"
    os.makedirs(ct_folder, exist_ok=True)
    with zipfile.ZipFile(fname, 'r') as zip_ref:
        zip_ref.extractall(ct_folder)

# Step 4: List contents of both folders
print("\n📂 Sinogram files:")
!find /content/sinograms -name '*.jpg' | head -n 10

print("\n📂 CT image files:")
!find /content/ct_images -name '*.jpg' | head -n 10


🔼 Upload your Sinogram ZIP (e.g. FILTERED_BRAIN_CT_SINOGRAMS.zip)


Saving FILTERED_BRAIN_CT_SINOGRAMS.zip.zip to FILTERED_BRAIN_CT_SINOGRAMS.zip (8).zip

🔼 Upload your CT Image ZIP (e.g. Brain_CT.zip)


Saving Brain_CT.zip.zip to Brain_CT.zip (8).zip

📂 Sinogram files:
/content/sinograms/FILTERED_BRAIN_CT_SINOGRAMS/._120 (5).jpg
/content/sinograms/FILTERED_BRAIN_CT_SINOGRAMS/._114 (17).jpg
/content/sinograms/FILTERED_BRAIN_CT_SINOGRAMS/._95 (4) - Copy - Copy.jpg
/content/sinograms/FILTERED_BRAIN_CT_SINOGRAMS/._120 (23).jpg
/content/sinograms/FILTERED_BRAIN_CT_SINOGRAMS/._123 (23).jpg
/content/sinograms/FILTERED_BRAIN_CT_SINOGRAMS/._100 (12).jpg
/content/sinograms/FILTERED_BRAIN_CT_SINOGRAMS/._110 (4).jpg
/content/sinograms/FILTERED_BRAIN_CT_SINOGRAMS/._60 (27).jpg
/content/sinograms/FILTERED_BRAIN_CT_SINOGRAMS/._62 (12).jpg
/content/sinograms/FILTERED_BRAIN_CT_SINOGRAMS/._128 (15).jpg

📂 CT image files:
/content/ct_images/Brain_CT/60 (6).jpg
/content/ct_images/Brain_CT/104 (19).jpg
/content/ct_images/Brain_CT/96 (33) - Copy - Copy.jpg
/content/ct_images/Brain_CT/62 (4).jpg
/content/ct_images/Brain_CT/106 (4).jpg
/content/ct_images/Brain_CT/130 (2).jpg
/content/ct_images/Brain_CT/51 (5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
